<a href="https://colab.research.google.com/github/ClaudiaMarano/Anomaly-Detection-and-Prediction/blob/main/network_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Physical Anomaly Detection

## Import Librerie

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.callbacks import Callback
from keras.optimizers import Adam
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.layers import LeakyReLU
from keras.layers import Dropout

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


## Caricamento e Preprocessing
Il dataset viene suddiviso in intervalli di un minuto in modo da addestrare la rete su intervalli di campioni senza anomalie, in modo da poter riconoscere in fase di test quando un intervallo contiene invece un'anomalia.


### Caricamento e Divisione Dataset

Il dataset viene suddiviso in segmenti di 200 paccheti in modo da addestrare la rete su intervalli di campioni senza anomalie, in modo da poter riconoscere in fase di test quando un intervallo contiene invece un'anomalia.

In [53]:
def load_and_split_data(path):
    # Caricamento del dataset
    df = pd.read_csv(path, encoding="utf-8")

    # Assicuriamoci che 'Time' sia in formato datetime
    df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')

    # Rimuovi righe con valori di data non validi
    df = df.dropna(subset=['Time'])

    # Ordinare i dati per timestamp
    df_normal = df.sort_values(by='Time')

    # Divisione in intervalli di un minuto
    segments = []
    window_duration = pd.Timedelta(milliseconds=100)
    start_time = df_normal['Time'].iloc[0]

    print("Start Time: ", df_normal['Time'].iloc[0])
    print("Finish Time: ", df_normal['Time'].iloc[-1])

    while start_time < df_normal['Time'].iloc[-1]:
      end_time = start_time + window_duration
      segment = df_normal[(df_normal['Time'] >= start_time) & (df_normal['Time'] < end_time)]
      if len(segment) > 0:
        print(segment.iloc[0])
        segments.append(segment.drop(columns=['Time', 'label', 'label_n', 'modbus_response']).reset_index(drop=True).values)
      start_time = end_time

    print(type(segments[0]))

    # Filtra i segmenti per ottenere solo quelli esattamente di 200 righe
    valid_segments = []
    for segment in segments:
        if len(segment) > 200:
            # Mantieni solo le prime 200 righe
            valid_segments.append(segment[:200])
        elif len(segment) == 200:
            # Segmento già valido
            valid_segments.append(segment)

    # print(f"Number of valid segments: {len(valid_segments)}")

    return valid_segments

### Preprocessing Dataset

Il preprocessing avviene come segue:
  
  1) Accorpo tutti i segmenti in un unico numpy array per poter applicare la normalizzazione.

  2) Normalizzo.

  3) Divido di nuovo i segmenti normalizzati nel numero originario dei segmenti in input.

  4) Ritorno i segmenti normalizzati e lo scaler utilizzato.

In [3]:
# 2.
def preprocessing(segments):
    """
    :param segments:
    :return:
    """

    # Preprocessing delle feature
    # Separiamo colonne categoriali e numeriche

    categorical_columns = [0, 1, 2, 3, 6, 7, 9]
    numerical_columns = [4, 5, 8, 10, 11]

    # 1)
    segments_array = np.vstack(segments)

    # print(f"Array Unico: {segments_array}")
    # print(f"Tipologia: {type(segments_array)}")
    # print(f"Lunghezza: {len(segments_array)}")
    # print("\n---------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n")

    # 2)
    # Configura il preprocessore con categorie globali e scaling numerico
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(categories='auto', handle_unknown='ignore'), categorical_columns),
            ('num', MinMaxScaler(), numerical_columns)
        ]
    )

    segments_scaled = preprocessor.fit_transform(segments_array)

    # print(f"Segmenti Normalizzati: {segments_scaled[0:1]}")
    # print(f"Tipologia: {type(segments_scaled)}")
    # print(f"Lunghezza: {segments_scaled.shape}")
    # print("\n---------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n")

    # 3)
    segments_scaled_split = np.array_split(segments_scaled.toarray(), len(segments))

    # print(f"Segmenti Normalizzati e Ricostruiti: {segments_scaled_split}")
    # print(f'Tipologia: {type(segments_scaled_split)}')
    # print(f"Lunghezza: {len(segments_scaled_split)}")
    # print("\n---------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n")

    # 4)
    return segments_scaled_split, preprocessor

## Funzione per il Training

In [9]:
# 3.
def building_and_training(segments_scaled_split):
    # Callback per visualizzare statistiche al termine dell'addestramento
    class TrainingSummary(Callback):
        def on_train_end(self, logs=None):
            print("\n--- Statistiche Finali ---")
            print(f"Loss finale su training set: {logs['loss']:.4f}")
            if 'val_loss' in logs:
                print(f"Loss finale su validation set: {logs['val_loss']:.4f}")

    # Definisco l'input shape
    input_shape = segments_scaled_split[0].shape  # Forma di un segmento

    # Definisco la struttura dell'autoencoder
    input_layer = Input(shape=input_shape)
    x = Flatten()(input_layer)
    x = Dense(256)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)  # Dropout 10%
    x = Dense(128)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)  # Dropout 10%
    encoded = Dense(64)(x)
    encoded = LeakyReLU(alpha=0.1)(encoded)
    x = Dense(128)(encoded)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)  # Dropout 10%
    x = Dense(256)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)  # Dropout 10%
    x = Dense(np.prod(input_shape), activation="sigmoid")(x)
    x = BatchNormalization()(x)
    decoded = Reshape(input_shape)(x)

    # Creo il Modello
    autoencoder = Model(inputs=input_layer, outputs=decoded)
    autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss="mse")

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    # Addestramento dell'autoencoder sui dati "normali", senza anomalie
    history = autoencoder.fit(
        np.array(segments_scaled_split), np.array(segments_scaled_split),
        epochs=1000, batch_size=16, shuffle=True, validation_split=0.1,
        callbacks=[TrainingSummary(), early_stopping]
    )

    # Stampo statistiche finali direttamente dal dizionario `history.history`

    print("\n--- Risultati Finali ---")
    print(f"Training Loss: {history.history['loss'][-1]:.4f}")
    print(f"Validation Loss: {history.history['val_loss'][-1]:.4f}")

    return autoencoder


## Calcolo dell'Errore di Ricostruzione

In [5]:
# 4.
def get_rebuilding_error(autoencoder, segments_scaled_split):
    # Controlla il tipo e il formato dei segmenti
    print(f"Tipo originale: {type(segments_scaled_split)}")
    print(f"Esempio di segmento: {type(segments_scaled_split[0])}, forma: {segments_scaled_split[0].shape}")

    # Converte ogni segmento in float32 se necessario
    segments_scaled_split = [segment.astype(np.float32) for segment in segments_scaled_split]

    # Converte la lista in un array tridimensionale
    segments_array = np.array(segments_scaled_split)
    print(f"Forma dell'array tridimensionale: {segments_array.shape}")

    # Verifica il tipo dei dati
    if not np.issubdtype(segments_array.dtype, np.floating):
        raise ValueError("I dati devono essere di tipo float32 o float64.")

    # Predizione della ricostruzione
    reconstructed_train = autoencoder.predict(segments_array)

    # Calcola l'errore di ricostruzione
    mse_train = np.mean(np.power(segments_array - reconstructed_train, 2), axis=(1, 2))

    # Calcola il threshold
    threshold = np.percentile(mse_train, 95)
    print("Soglia di errore di ricostruzione:", threshold)

    return threshold

## Test del modello

In [58]:
def load_and_split_data_test(path):
    # Caricamento del dataset
    df = pd.read_csv(path, encoding="utf-8")

    # Assicuriamoci che 'Time' sia in formato datetime
    df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')

    # Rimuovi righe con valori di data non validi
    df = df.dropna(subset=['Time'])

    # Ordinare i dati per timestamp
    df_normal = df.sort_values(by='Time')

    # Divisione in intervalli di un minuto
    segments = []
    window_duration = pd.Timedelta(milliseconds=100)
    start_time = df_normal['Time'].iloc[0]

    print("Start Time: ", df_normal['Time'].iloc[0])
    print("Finish Time: ", df_normal['Time'].iloc[-1])

    while start_time < df_normal['Time'].iloc[-1]:
      end_time = start_time + window_duration
      segment = df_normal[(df_normal['Time'] >= start_time) & (df_normal['Time'] < end_time)]
      if len(segment) > 0:
        # segments.append(segment.drop(columns=['Time', 'label', 'label_n', 'modbus_response']).reset_index(drop=True).values)
        segments.append(segment.drop(columns=['Time']).reset_index(drop=True).values)
      start_time = end_time

    print(type(segments[0]))

    # Filtra i segmenti per ottenere solo quelli esattamente di 200 righe
    valid_segments = []
    for segment in segments:
        if len(segment) > 200:
            # Mantieni solo le prime 200 righe
            valid_segments.append(segment[:200])
        elif len(segment) == 200:
            # Segmento già valido
            valid_segments.append(segment)

    # print(f"Number of valid segments: {len(valid_segments)}")

    return valid_segments

In [70]:
def test_model(autoencoder, test_data_path, preprocessor, threshold):
    """
    Testa l'autoencoder su un dataset di test e identifica segmenti anomali.

    :param autoencoder: Modello autoencoder addestrato.
    :param test_data_path: Path al dataset di test.
    :param preprocessor: Oggetto `ColumnTransformer` usato per il preprocessing.
    :param threshold: Soglia di errore di ricostruzione per identificare anomalie.
    :return: Dizionario con errore medio di ricostruzione e anomalie rilevate.
    """
    # 1. Carica e dividi il dataset in segmenti
    test_segments = load_and_split_data_test(test_data_path)
    print("Numero di segmenti:", len(segments))
    print(segments[0].shape)

    # 2. Preprocessing (trasformazione dei dati)
    # test_segments_array = np.vstack(test_segments)
    # test_segments_scaled = preprocessor.transform(test_segments_array)
    # test_segments_scaled_split = np.array_split(test_segments_scaled, len(test_segments))

    processed_segments, preprocessor = preprocessing(test_segments)

    # 3. Predizione con l'autoencoder
    reconstructed_test = autoencoder.predict(np.array(processed_segments))

    # 4. Calcolo dell'errore di ricostruzione
    mse_test = np.mean(np.power(np.array(processed_segments) - reconstructed_test, 2), axis=(1, 2))

    # 5. Identificazione delle anomalie
    anomalies = mse_test > threshold

    # 6. Conteggio delle anomalie
    anomaly_count = np.sum(anomalies)

    return {
        "mse": mse_test,
        "anomalies": anomalies,
        "anomaly_count": anomaly_count
    }

## Esecuzione del processo

In [36]:
segments = load_and_split_data("normal_reduced_0005.csv")
print("Numero di segmenti:", len(segments))
print(segments[0].shape)

Start Time:  2021-04-09 11:30:52.716203
Finish Time:  2021-04-09 11:30:54.462284
<class 'numpy.ndarray'>
Numero di segmenti: 17
(200, 12)


In [37]:
processed_segments, preprocessor = preprocessing(segments)

In [10]:
autoencoder = building_and_training(processed_segments)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 0.9437 - val_loss: 0.2391
Epoch 2/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 651ms/step - loss: 0.9204 - val_loss: 0.2375
Epoch 3/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - loss: 0.8623 - val_loss: 0.2359
Epoch 4/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - loss: 0.7840 - val_loss: 0.2343
Epoch 5/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - loss: 0.7242 - val_loss: 0.2328
Epoch 6/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - loss: 0.6657 - val_loss: 0.2314
Epoch 7/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - loss: 0.6183 - val_loss: 0.2300
Epoch 8/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - loss: 0.5805 - val_loss: 0.2288
Epoch 9/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - loss: 0.5797 - val_loss: 0.2278
Epoch 10/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - loss: 0.5611 - val_loss: 0.2267
Epoch 11/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - loss: 0.5619 - val_loss: 0.2256
Epoch 12/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - 

In [38]:
threshold = get_rebuilding_error(autoencoder, processed_segments)

Tipo originale: <class 'list'>
Esempio di segmento: <class 'numpy.ndarray'>, forma: (200, 47)
Forma dell'array tridimensionale: (17, 200, 47)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Soglia di errore di ricostruzione: 0.085909765958786


In [72]:
test_results = test_model(
    autoencoder=autoencoder,
    test_data_path="attack_1_0005.csv",
    preprocessor=preprocessor,
    threshold=threshold
)

# Visualizzazione dei risultati
print("Errori di ricostruzione (MSE):", test_results["mse"], "\n")
print("Anomalie rilevate:", test_results["anomalies"], "\n")
print("Numero totale di anomalie:", test_results["anomaly_count"])

Start Time:  2021-04-09 18:23:28.385003
Finish Time:  2021-04-09 18:23:29.618596
<class 'numpy.ndarray'>
Numero di segmenti: 17
(200, 12)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Errori di ricostruzione (MSE): [0.09794884 0.09215064 0.11308716 0.09603768 0.09801166 0.10030539
 0.10035264 0.11253008 0.11003965 0.11931071 0.10299337 0.10246325] 

Anomalie rilevate: [ True  True  True  True  True  True  True  True  True  True  True  True] 

Numero totale di anomalie: 12
